In [1]:
import torch
import model

In [2]:
model = model.MiniConvNet()
model.load_state_dict(torch.load("model.pkl", map_location='cpu'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [3]:
for name,parameters in model.named_parameters():
    print(name,':',parameters.size())

features.0.weight : torch.Size([32, 1, 3, 3])
features.1.weight : torch.Size([32])
features.1.bias : torch.Size([32])
features.5.weight : torch.Size([32, 32, 3, 3])
features.6.weight : torch.Size([32])
features.6.bias : torch.Size([32])
features.9.weight : torch.Size([32, 32, 3, 3])
features.10.weight : torch.Size([32])
features.10.bias : torch.Size([32])
features.14.weight : torch.Size([32, 32, 3, 3])
features.15.weight : torch.Size([32])
features.15.bias : torch.Size([32])
classifier.0.weight : torch.Size([20, 1568])
classifier.1.weight : torch.Size([20])
classifier.1.bias : torch.Size([20])
classifier.4.weight : torch.Size([10, 20])


In [4]:
for name, sub_module in model.named_modules():
    if type(sub_module).__base__ is torch.nn.Conv2d:
        print(name)
        print(sub_module.bias)

features.0
None
features.5
None
features.9
None
features.14
None


# 生成配置头文件

In [5]:
def write_define(define_name, define_value, config_file):
    config_file.write("#define "+str(define_name)+" "+str(define_value)+"\n")
    
def generateConfig(model, in_row, in_col, path_to_config):
    config_file = open(path_to_config, "w")
    cov_cnt = 0
    pool_cnt = 0
    linear_cnt = 0
    in_ch = 0
    for name, sub_module in model.named_modules():
        if type(sub_module).__base__ is torch.nn.Conv2d:
            write_define("COV_" + str(cov_cnt) + "_IN_CH", sub_module.in_channels, config_file)
            write_define("COV_" + str(cov_cnt) + "_IN_ROW", in_row, config_file)
            write_define("COV_" + str(cov_cnt) + "_IN_COL", in_col, config_file)
            
            in_row = (in_row + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            in_col = (in_col + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            in_ch = sub_module.out_channels
            write_define("COV_" + str(cov_cnt) + "_OUT_CH", sub_module.out_channels, config_file)
            write_define("COV_" + str(cov_cnt) + "_OUT_ROW", in_row, config_file)
            write_define("COV_" + str(cov_cnt) + "_OUT_COL", in_col, config_file)
            
           
            write_define("COV_" + str(cov_cnt) + "_K", sub_module.kernel_size[0], config_file)
            write_define("COV_" + str(cov_cnt) + "_S", sub_module.stride[0], config_file)
            write_define("COV_" + str(cov_cnt) + "_P", sub_module.padding[0], config_file)
            if sub_module.bias != None:
                write_define("COV_" + str(cov_cnt) + "_B", 1, config_file)
            else:
                write_define("COV_" + str(cov_cnt) + "_B", 0, config_file)
            cov_cnt = cov_cnt + 1
            config_file.write("\n")
        elif type(sub_module) is torch.nn.MaxPool2d:
            write_define("POOL_" + str(pool_cnt) + "_IN_CH", in_ch, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_ROW", in_row, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_COL", in_col, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_PO", sub_module.kernel_size, config_file)
            in_row = in_row // sub_module.kernel_size
            in_col = in_col // sub_module.kernel_size
            pool_cnt = pool_cnt + 1
            config_file.write("\n")
        elif type(sub_module).__base__ is torch.nn.Linear:
            write_define("LINEAR_" + str(linear_cnt) + "_IN_N", sub_module.in_features, config_file)
            write_define("LINEAR_" + str(linear_cnt) + "_OUT_N", sub_module.out_features, config_file)
            linear_cnt = linear_cnt + 1
            config_file.write("\n")
#         elif type(sub_module) is torch.nn.ReLU:
#             write_define("LINEAR_" + str(linear_cnt) + "_IN_N", sub_module.in_features, config_file)
    config_file.close()

In [6]:
generateConfig(model, 28, 28, 'config.h')

# 生成参数

In [16]:
import numpy as np
import quant_dorefa

def weight_quant(weight, bit):
    quantize_fn = quant_dorefa.weight_quantize_fn(w_bit=bit)
    w = quantize_fn(weight)
#     print(w)
    w = w.detach().numpy()
    w = (w + 1) / 2 * (2**bit - 1) + 0.5
    w = w.astype(np.int8)
    return w

def generate_params(model):
    BIT = 2
    
    cov_cnt = 0
    linear_cnt = 0
    bn_cnt = 0

    for name, sub_module in model.named_modules():
        if type(sub_module).__base__ is torch.nn.Conv2d:
            w = sub_module.weight
#             print(w)
            w = weight_quant(w, BIT)
#             print(w)
            w.tofile('prama/' + 'conv_' + str(cov_cnt) + '_w.bin')
            if sub_module.bias != None:
                b = sub_module.bias.detach().numpy()
                b.tofile('prama/' + 'conv_' + str(cov_cnt) + '_b.bin')
            cov_cnt = cov_cnt + 1    
        elif type(sub_module).__base__ is torch.nn.Linear:
            w = sub_module.weight
#             print(w)
            w = weight_quant(w, BIT)
#             print(w)
            w.tofile('prama/' + 'linear_' + str(linear_cnt) + '_w.bin')
            if sub_module.bias != None:
                b = sub_module.bias.detach().numpy()
                b.tofile('prama/' + 'linear_' + str(linear_cnt) + '_b.bin')          
            linear_cnt = linear_cnt + 1
        elif type(sub_module) is torch.nn.BatchNorm2d or type(sub_module) is torch.nn.BatchNorm1d:
            gamma = sub_module.weight.detach().numpy()
            var = sub_module.running_var.numpy()
            mean = sub_module.running_mean.numpy()
            eps = sub_module.eps
            beta = sub_module.bias.detach().numpy()
               
            w = gamma / (np.sqrt(var) + eps)
            w.tofile('prama/' + 'bn_' + str(bn_cnt) + '_w.bin')

            
            b = beta - (mean / (np.sqrt(var) + eps) * gamma)
#             print(eps)
            print('beta = ', beta)
            print('b = ', b)
            b.tofile('prama/' + 'bn_' + str(bn_cnt) + '_b.bin') 
            bn_cnt = bn_cnt + 1

generate_params(model)

beta =  [-7.40166893e-03  2.28819661e-02  1.20181516e-02  1.53189525e-02
  9.49030649e-03 -1.07677110e-01 -6.25235736e-02  3.87686379e-02
 -2.00911704e-02 -5.85516542e-03 -3.91086005e-02  1.64073501e-02
  1.51579753e-02 -2.27795187e-02 -1.06302686e-01 -2.14228071e-02
 -4.99702953e-02  1.15341321e-02  5.86927496e-02 -1.26750886e-01
 -4.15025204e-02 -1.08206952e-02  2.61574406e-02  3.55197638e-02
  7.63492957e-02 -2.94177170e-07  5.91052230e-03 -6.58446327e-02
 -5.19776344e-02  1.26479074e-01 -3.71805541e-02 -6.68370277e-02]
b =  [ 0.11771239  0.07446921  0.4286759   0.16201591  0.4192319  -0.20432132
 -0.07214875 -0.13055389 -0.33874443 -0.2086187  -0.39286214  0.11427573
  0.42514944 -0.02504103 -0.25834054  0.31159416 -0.02778467  0.21007025
  0.11746836 -0.3005185  -0.14851207 -0.34213322  0.03892953  0.21747239
  0.12901337  0.14157896  0.25838727 -0.20128712  0.3894534   0.16819951
 -0.09720761 -0.2807455 ]
beta =  [-0.00637998 -0.01419008  0.01947843 -0.06865593  0.00966713 -0.065

In [8]:
a = np.array([1.3, 2.4, 3.5, 4.9])
print(a = 1)
print(a.astype(np.int8))

TypeError: 'a' is an invalid keyword argument for this function